In [2]:
import pandas as pd
import numpy as np

np.random.seed(77)

# Synthetic dataset
n = 1500
genders = ["Male", "Female", "Other"]
cities = ["Metro", "Urban", "Town"]

data = pd.DataFrame({
    "Driver_ID": range(1, n+1),
    "Age": np.random.randint(21, 65, n),
    "Gender": np.random.choice(genders, n, p=[0.6, 0.35, 0.05]),
    "City_Type": np.random.choice(cities, n, p=[0.4, 0.45, 0.15]),
    "Trips_Completed": np.random.randint(50, 2000, n),
    "Avg_Rating": np.round(np.random.uniform(3.0, 5.0, n), 2),
    "Complaints": np.random.randint(0, 20, n),
    "Accidents": np.random.randint(0, 5, n)
})

# Introduce base label
data["Flagged_For_Review"] = np.where(
    (data["Complaints"] > 5) | (data["Accidents"] > 0),
    np.random.choice([1, 0], n, p=[0.7, 0.3]),
    np.random.choice([1, 0], n, p=[0.15, 0.85])
)

# Add bias: higher flag rates for Metro drivers
mask_metro = data["City_Type"] == "Metro"
data.loc[mask_metro, "Flagged_For_Review"] = np.random.choice([1, 0], mask_metro.sum(), p=[0.4, 0.6])

data.head()

,Driver_ID,Age,Gender,City_Type,Trips_Completed,Avg_Rating,Complaints,Accidents,Flagged_For_Review
0,1,44,Other,Metro,324,3.88,12,3,1
1,2,52,Male,Urban,323,3.96,17,3,0
2,3,41,Female,Urban,448,3.53,4,4,1
3,4,41,Female,Town,1485,4.83,19,1,0
4,5,64,Male,Urban,501,3.66,15,2,0


Task 1 — Bias Assessment Before Deployment

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare data (one-hot encoding for categorical vars)
X = pd.get_dummies(data.drop(columns=["Driver_ID", "Flagged_For_Review"]), drop_first=True)
y = data["Flagged_For_Review"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)
print("Baseline Accuracy:", round(accuracy_score(y_test, y_pred), 3))

# Merge sensitive attrs for bias check
df_test = X_test.copy()
df_test["y_pred"] = y_pred
df_test["Gender"] = data.loc[y_test.index, "Gender"]
df_test["City_Type"] = data.loc[y_test.index, "City_Type"]

# Bias check
print("\nPositive prediction rate by Gender:")
print(df_test.groupby("Gender")["y_pred"].mean())

print("\nPositive prediction rate by City_Type:")
print(df_test.groupby("City_Type")["y_pred"].mean())

Baseline Accuracy: 0.589

Positive prediction rate by Gender:
Gender
Female    0.453901
Male      0.583333
Other     0.619048
Name: y_pred, dtype: float64

Positive prediction rate by City_Type:
City_Type
Metro    0.186170
Town     0.808219
Urban    0.798942
Name: y_pred, dtype: float64


Task 2 — Ethics‑Aware Fairness Gate

In [4]:
def fairness_check(df, sensitive_attr, threshold=0.08):
    rates = df.groupby(sensitive_attr)["y_pred"].mean()
    disparity = rates.max() - rates.min()
    print(f"{sensitive_attr} disparity: {round(disparity, 3)}")
    return disparity <= threshold

print("\nFairness Gate Results:")
gender_fair = fairness_check(df_test, "Gender")
city_fair = fairness_check(df_test, "City_Type")

if not (gender_fair and city_fair):
    print("❌ Fairness gate FAILED — mitigation needed before deployment.")
else:
    print("✅ Fairness gate PASSED")


Fairness Gate Results:
Gender disparity: 0.165
City_Type disparity: 0.622
❌ Fairness gate FAILED — mitigation needed before deployment.


Task 3 — Feature Importance Ethics Audit

In [5]:
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop Features by Importance:")
print(importances.head(10))


Top Features by Importance:
Avg_Rating         0.226965
Trips_Completed    0.221725
Age                0.175344
Complaints         0.154684
Accidents          0.087626
City_Type_Urban    0.058496
Gender_Male        0.036379
City_Type_Town     0.028814
Gender_Other       0.009967
dtype: float64


Task 4 — Ethics Development User Stories
Two sample user stories:

US1: As an ethics officer, I want the driver evaluation model to maintain bias disparity below 8% so that reviews are fair across all demographics.
US2: As an operations manager, I want all model predictions logged with context so any disputed driver flagging can be audited.

Task 5 — Accountability via Prediction Logging

In [6]:
import datetime

prediction_log = []
MODEL_VERSION = "v1.0"

def predict_and_log(driver_features):
    pred = model.predict(driver_features)[0]
    log_entry = {
        "timestamp": datetime.datetime.now().isoformat(),
        "model_version": MODEL_VERSION,
        "inputs": driver_features.to_dict(orient='records')[0],
        "prediction": int(pred)
    }
    prediction_log.append(log_entry)
    return pred

# Example log
sample_driver = X_test.iloc[[0]]
predict_and_log(sample_driver)
prediction_log[:1]

[{'timestamp': '2025-08-13T08:32:02.007902',
  'model_version': 'v1.0',
  'inputs': {'Age': 56,
   'Trips_Completed': 1167,
   'Avg_Rating': 3.95,
   'Complaints': 9,
   'Accidents': 3,
   'Gender_Male': True,
   'Gender_Other': False,
   'City_Type_Town': False,
   'City_Type_Urban': True},
  'prediction': 1}]

Task 6 — Post‑Deployment Monitoring

In [7]:
# Demographic parity after deployment
dp_gender = df_test.groupby("Gender")["y_pred"].mean().max() - df_test.groupby("Gender")["y_pred"].mean().min()
dp_city = df_test.groupby("City_Type")["y_pred"].mean().max() - df_test.groupby("City_Type")["y_pred"].mean().min()

print(f"Gender disparity: {dp_gender:.3f}")
print(f"City disparity: {dp_city:.3f}")

if dp_gender > 0.08 or dp_city > 0.08:
    print("⚠️ ALERT: Bias detected — review required")
else:
    print("✅ Fairness levels acceptable")

Gender disparity: 0.165
City disparity: 0.622
⚠️ ALERT: Bias detected — review required


Task 7 — Privacy Protection

In [8]:
import hashlib

data["Driver_ID"] = data["Driver_ID"].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())
data.head()

,Driver_ID,Age,Gender,City_Type,Trips_Completed,Avg_Rating,Complaints,Accidents,Flagged_For_Review
0,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,44,Other,Metro,324,3.88,12,3,1
1,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,52,Male,Urban,323,3.96,17,3,0
2,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,41,Female,Urban,448,3.53,4,4,1
3,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,41,Female,Town,1485,4.83,19,1,0
4,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,64,Male,Urban,501,3.66,15,2,0


Task 8 — Inclusivity Feedback Loop

In [9]:
# Sample feedback from underrepresented groups
feedback = pd.DataFrame({
    "Age": [30, 45],
    "Gender": ["Other", "Other"],
    "City_Type": ["Town", "Town"],
    "Trips_Completed": [500, 1200],
    "Avg_Rating": [4.6, 4.3],
    "Complaints": [1, 0],
    "Accidents": [0, 0],
    "Flagged_For_Review": [0, 0]
})

# Append for retraining
data_updated = pd.concat([data, feedback], ignore_index=True)
print("Updated dataset size:", data_updated.shape)

Updated dataset size: (1502, 9)
